# **RAG Fusion**

1.	クエリからLLMでサブワード（サブクエリ）生成
　LLMを使って、元のクエリから複数の角度や細分化したサブワード（サブクエリ）を作り出す。
　これにより、クエリの多面的な意味が捉えられる。
2.	各サブワードに対するコンテキスト文章の検索
　それぞれのサブワードを使って、関連性の高いコンテキスト文章を検索する。
　ここでは、類似度スコア（例えば、BM25やコサイン類似度など）を用いて、各文章を順序付けする。
3.	RRFでのスコア付け
　そして、各サブワードごとに得られた順位情報を、RRF（Reciprocal Rank Fusion）という手法で重み付けして、各コンテキスト文章の最終的なスコアを算出する。これは単に順位が低いほうが分母が大きくなるのでスコアが低くなるという計算だ。「1 / (ランク + 定数)」

これにより、複数のサブワード（サブクエリ）からの情報が統合され、より正確なランキングが得られる。

Research Paper: [RAG Fusion](https://arxiv.org/pdf/2402.03367)

実際のシステムでは定数は60など大きい値が使われることが多いが、ここでは計算を分かりやすくするために定数を 1 としてみる。

'''
サブクエリ1でのスコア計算
	•	文書A：
　ランク = 1 → スコア = 1 / (1 + 1) = 1/2 = 0.5
	•	文書B：
　ランク = 2 → スコア = 1 / (2 + 1) = 1/3 ≈ 0.333
	•	文書C：
　ランク = 4 → スコア = 1 / (4 + 1) = 1/5 = 0.2

サブクエリ2でのスコア計算
	•	文書B：
　ランク = 1 → スコア = 1 / (1 + 1) = 1/2 = 0.5
	•	文書C：
　ランク = 2 → スコア = 1 / (2 + 1) = 1/3 ≈ 0.333
	•	文書A：
　ランク = 3 → スコア = 1 / (3 + 1) = 1/4 = 0.25

合計スコアの算出

各サブクエリで得られたスコアを文書ごとに足し合わせる：
	•	文書A：
　サブクエリ1: 0.5 ＋ サブクエリ2: 0.25 = 0.75
	•	文書B：
　サブクエリ1: 0.333 ＋ サブクエリ2: 0.5 = 0.833
	•	文書C：
　サブクエリ1: 0.2 ＋ サブクエリ2: 0.333 = 0.533

最終ランク付け

合計スコアが高い順にランク付けすると、最終的に以下のようになる。
	1.	文書B（0.833）
	2.	文書A（0.75）
	3.	文書C（0.533）
'''

In [1]:
# set apikey to environ
import os
from dotenv import load_dotenv
load_dotenv()
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")

In [2]:
from langchain_community.document_loaders import PyPDFLoader
file_path = "../data/pdf/57_public_スタートアップ育成に向けた政府の取組_file_name=kaisetsushiryou_2024.pdf"
loader = PyPDFLoader(file_path)
documents = loader.load_and_split()
documents[0]

Document(metadata={'source': '../data/pdf/57_public_スタートアップ育成に向けた政府の取組_file_name=kaisetsushiryou_2024.pdf', 'page': 0, 'page_label': '1'}, page_content='スタートアップの力で\n社会課題解決 と経済成長 を加速する\nスタートアップ育成に向けた政府の取組\n2024年9月')

In [3]:
# load embedding model
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

# split pages content
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

# load vectorstore
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents, embeddings)

# create retriever
retriever = vectorstore.as_retriever()

Reciprocal Rank Fusion Chain

In [4]:
# create llm
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

# 最適化されたRAG Fusion用のプロンプトテンプレートを取得しています
from langchain_core.output_parsers import StrOutputParser
from langsmith import Client
client = Client()
prompt = client.pull_prompt("langchain-ai/rag-fusion-query-generation")
print(prompt)

input_variables=['original_query'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'langchain-ai', 'lc_hub_repo': 'rag-fusion-query-generation', 'lc_hub_commit_hash': '478b448e096b977446865108fad34282e6e1a84ae8b8540572ed0df238229a11'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant that generates multiple search queries based on a single input query.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['original_query'], input_types={}, partial_variables={}, template='Generate multiple search queries related to: {original_query}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='OUTPUT (4 queries):'), additional_kwargs={})]


In [5]:
# generate sub queries
generate_queries = (
    prompt | ChatOpenAI(temperature=0) | StrOutputParser() | (lambda x: x.split("\n"))
) # RAG Fusionでは複数のサブクエリを個別に扱う必要があります split("\n")を使うことで、改行で区切られた文字列を配列（リスト）に変換します
# "クエリ1\nクエリ2\nクエリ3" -> ["クエリ1", "クエリ2", "クエリ3"]

In [6]:
# rerank results
from langchain.load import dumps, loads
from collections import defaultdict

def reciprocal_rank_fusion(results: list[list], k=60):
    # defaultdictで初期値0を自動設定
    fused_scores = defaultdict(float)
    
    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            fused_scores[doc_str] += 1 / (rank + k)
    
    reranked_results = [
        (loads(doc), score) 
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results

In [7]:
# create chain
# map()メソッドは、複数のサブクエリそれぞれに対してretrieverを適用するために使用されています
# 例: サブクエリが["クエリ1", "クエリ2"]の場合、
# [retriever("クエリ1"), retriever("クエリ2")]のように各クエリに対して検索を実行します
chain = generate_queries | retriever.map() | reciprocal_rank_fusion

# check input schema
chain.input_schema.schema()

/var/folders/yl/5c7mgh4j7j9clvvpsvwjbvhr0000gn/T/ipykernel_15507/1342569176.py:8: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  chain.input_schema.schema()


{'properties': {'original_query': {'title': 'Original Query',
   'type': 'string'}},
 'required': ['original_query'],
 'title': 'PromptInput',
 'type': 'object'}

In [11]:
# test 
test_results = chain.invoke("スタートアップによる経済的効果は?")
print(test_results)

[(Document(metadata={'page': 31, 'page_label': '32', 'source': '../data/pdf/57_public_スタートアップ育成に向けた政府の取組_file_name=kaisetsushiryou_2024.pdf'}, page_content='３．スタートアップのための資金供給の強化と出口戦略の多様化'), 0.06666666666666667), (Document(metadata={'page': 18, 'page_label': '19', 'source': '../data/pdf/57_public_スタートアップ育成に向けた政府の取組_file_name=kaisetsushiryou_2024.pdf'}, page_content='２．スタートアップ創出に向けた人材・ネットワークの構築'), 0.06504494976203068), (Document(metadata={'page': 3, 'page_label': '4', 'source': '../data/pdf/57_public_スタートアップ育成に向けた政府の取組_file_name=kaisetsushiryou_2024.pdf'}, page_content='\uf070 スタートアップによるGDP創出額は、直接効果で10.47兆円、間接波及効果を含めると19.39兆円と試算。\n\uf070 直接効果は通信・放送業の名目GDPに、間接波及効果を含めた値は北海道内の名目GDPに相当しており、\n一定の経済的インパクトを発揮していると言える。\nスタートアップはマクロ経済に一定のインパクトを与える\nスタートアップによる経済効果\n*1. 直接効果とは、スタートアップの経済活動により創出される付加価値を指す ( 産業連関表を用いた数値\nではない )\n*2. 間接波及効果とは、スタートアップに対するサプライヤーの経済活動や所得創出に伴う消費支出が引き\n金となり連鎖的に創出される経済効果を指す。本調査では産業連関表を用いて 2 次波及効果まで推計\n*3. 県内名目GDPは内閣府の県民経済計算統計より経産省作成\nGDP創出のイメージ\n県内名目GDP（2020）との比較\n東京都\n大阪府\

# RAG Chain

In [12]:
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context.
If you don't find the answer in the context, just say that you don't know.

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

rag_fusion_chain = (
    {
        "context": chain,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
# test2 
test2_results = rag_fusion_chain.invoke("スタートアップによる経済的効果は?")
print(test2_results)

スタートアップによる経済的効果は、直接効果で10.47兆円、間接波及効果を含めると19.39兆円と試算されています。


In [15]:
# prepare data for evaluation
# test 
import pandas as pd
from datasets import load_dataset

# === データの読み込みとフィルタリング ===
test_data = load_dataset("allganize/RAG-Evaluation-Dataset-JA")["test"]

tgt_name = file_path.split("file_name=")[-1]  # どこかで定義済みの file_path
print(f"{tgt_name=}")

# 評価対象データだけを抽出
filtered_data = test_data.filter(lambda x: x["target_file_name"] == tgt_name)

# DataFrame 化して、全体の件数を確認
eval_df = filtered_data.to_pandas()
print(f"Length of eval_df: {len(eval_df)}")

# 必要な列だけ取り出す (質問と正解回答)
eval_df = eval_df[["question", "target_answer"]]

# === RAG チェーンを利用して回答と文脈を取得する ===
p_data = []  # 各サンプルごとの評価用データを貯めるリスト

for i, row in eval_df.iterrows():
    q = row["question"]
    t = row["target_answer"]
    
    # RAG チェーンを実行して回答を取得する
    # rag_fusion_chain や retriever はすでに定義済みのもの
    response_text = rag_fusion_chain.invoke(q)
    context_docs = retriever.invoke(q)
    
    p_data.append({
        "question": q,
        "target_answer": t,
        "response": response_text,
        "context": context_docs
    })

# === RAG 実行結果をまとめた新しい DataFrame を作成 ===
rag_eval_df = pd.DataFrame(p_data)
rag_eval_df.head()

tgt_name='kaisetsushiryou_2024.pdf'
Length of eval_df: 7


,question,target_answer,response,context
0,オープンイノベーション促進税制において、スタートアップ企業の株式取得に対する税制優遇措置は、...,オープンイノベーション促進税制の下で、新規発行株式の取得は「新規出資型」として分類され、発行...,"Based on the provided context, it is not possi...",[page_content=' 国内事業会社又はその国内CVCが、オープンイノベーションに...
1,イノベーション拠点税制における所得控除について、控除対象となる研究開発活動に関して具体的にど...,イノベーション拠点税制における所得控除の対象となるためには、企業が主に「国内で」「自ら」開発...,国内で自ら研究開発した知財に限る条件が求められます。,[page_content=' 我が国のイノベーション拠点の立地競争力を強化する観点から、...
2,「カーブアウト加速等支援事業」の主な目的は何ですか？,事業会社に蓄積されている技術を活用し、新たな会社を立ち上げた者や立ち上げる意思を持つ者に研究...,カーブアウト加速等支援事業の主な目的は、研究開発により得た技術を活用して新たな会社を立ち上げ...,[page_content=' 日本企業では、研究開発により得た技術であって、事業化されな...
3,グローバル・アクセラレーション・ハブの拠点がある北米の都市を全て教えてください。,グローバル・アクセラレーション・ハブの北米の拠点は、ボストン、ニューヨーク、シカゴ、オーステ...,シリコンバレー,[page_content=' 独立行政法人日本貿易振興機構（JETRO）\nは、世界８地...
4,産業革新投資機構がベンチャー・グロース・インベストメンツを通じて設立したファンドについて、資...,産業革新投資機構(JIC)は子会社であるベンチャー・グロース・インベストメンツ(VGI)を通...,"資金規模は2,000億円で、設立時期は2023年1月です。",[page_content=' 2023年１月、産業革新投資機構（JIC）の子会社であるベ...


In [17]:
import pandas as pd
from langchain.evaluation import load_evaluator
from langchain.evaluation import EvaluatorType

relevance_evaluator = load_evaluator(
    EvaluatorType.QA,
    criteria="relevance",
    model="gpt-4",
    model_kwargs={"temperature": 0}
)

accuracy_evaluator = load_evaluator(
    EvaluatorType.QA,
    criteria="accuracy",
    model="gpt-4",
    model_kwargs={"temperature": 0}
)

def evaluate_response(df):
    """
    与えられたデータリストに対する Relevance と Accuracy の評価を行い、結果を DataFrame で返します。
    """
    def _flatten_context(context_val):
        if isinstance(context_val, list):
            return "\n".join([item.page_content if hasattr(item, "page_content") else str(item) for item in context_val])
        return str(context_val)
    
    df["ContextText"] = df["context"].apply(lambda x: _flatten_context(x))

    def _evaluate_row(row):
        relevance_score = relevance_evaluator.evaluate_strings(
            prediction=row["response"],
            input=row["question"],
            reference=row["ContextText"]
        )
        accuracy_score = accuracy_evaluator.evaluate_strings(
            prediction=row["response"],
            input=row["question"],
            reference=row["target_answer"]
        )
        return relevance_score, accuracy_score
    
    df[["Relevance", "Accuracy"]] = df.apply(lambda row: pd.Series(_evaluate_row(row)), axis=1)
    
    df["RelevanceScore"] = df["Relevance"].apply(lambda x: 1 if x["value"].upper() == "CORRECT" else 0)
    df["AccuracyScore"] = df["Accuracy"].apply(lambda x: 1 if x["value"].upper() == "CORRECT" else 0)
    
    # 平均スコアを計算
    avg_relevance = df["RelevanceScore"].mean()
    avg_accuracy = df["AccuracyScore"].mean()
    
    print(f"Average Relevance: {avg_relevance:.2f}")
    print(f"Average Accuracy: {avg_accuracy:.2f}")

    return df

In [18]:
results = evaluate_response(rag_eval_df)
results 

Average Relevance: 0.57
Average Accuracy: 0.57


,question,target_answer,response,context,ContextText,Relevance,Accuracy,RelevanceScore,AccuracyScore
0,オープンイノベーション促進税制において、スタートアップ企業の株式取得に対する税制優遇措置は、...,オープンイノベーション促進税制の下で、新規発行株式の取得は「新規出資型」として分類され、発行...,"Based on the provided context, it is not possi...",[page_content=' 国内事業会社又はその国内CVCが、オープンイノベーションに..., 国内事業会社又はその国内CVCが、オープンイノベーションにより新事業開拓・生産性向上を図...,"{'reasoning': 'INCORRECT', 'value': 'INCORRECT...","{'reasoning': 'INCORRECT', 'value': 'INCORRECT...",0,0
1,イノベーション拠点税制における所得控除について、控除対象となる研究開発活動に関して具体的にど...,イノベーション拠点税制における所得控除の対象となるためには、企業が主に「国内で」「自ら」開発...,国内で自ら研究開発した知財に限る条件が求められます。,[page_content=' 我が国のイノベーション拠点の立地競争力を強化する観点から、..., 我が国のイノベーション拠点の立地競争力を強化する観点から、海外と比べて遜色ない事業環境の...,"{'reasoning': 'CORRECT', 'value': 'CORRECT', '...","{'reasoning': 'CORRECT', 'value': 'CORRECT', '...",1,1
2,「カーブアウト加速等支援事業」の主な目的は何ですか？,事業会社に蓄積されている技術を活用し、新たな会社を立ち上げた者や立ち上げる意思を持つ者に研究...,カーブアウト加速等支援事業の主な目的は、研究開発により得た技術を活用して新たな会社を立ち上げ...,[page_content=' 日本企業では、研究開発により得た技術であって、事業化されな..., 日本企業では、研究開発により得た技術であって、事業化されないものの多くが消滅しており、研...,"{'reasoning': 'CORRECT', 'value': 'CORRECT', '...","{'reasoning': 'CORRECT', 'value': 'CORRECT', '...",1,1
3,グローバル・アクセラレーション・ハブの拠点がある北米の都市を全て教えてください。,グローバル・アクセラレーション・ハブの北米の拠点は、ボストン、ニューヨーク、シカゴ、オーステ...,シリコンバレー,[page_content=' 独立行政法人日本貿易振興機構（JETRO）\nは、世界８地..., 独立行政法人日本貿易振興機構（JETRO）\nは、世界８地域・３０都市において、現地\n...,"{'reasoning': 'INCORRECT', 'value': 'INCORRECT...","{'reasoning': 'INCORRECT', 'value': 'INCORRECT...",0,0
4,産業革新投資機構がベンチャー・グロース・インベストメンツを通じて設立したファンドについて、資...,産業革新投資機構(JIC)は子会社であるベンチャー・グロース・インベストメンツ(VGI)を通...,"資金規模は2,000億円で、設立時期は2023年1月です。",[page_content=' 2023年１月、産業革新投資機構（JIC）の子会社であるベ..., 2023年１月、産業革新投資機構（JIC）の子会社であるベンチャー・グロース・インベスト...,"{'reasoning': 'CORRECT', 'value': 'CORRECT', '...","{'reasoning': 'CORRECT', 'value': 'CORRECT', '...",1,1
5,スタートアップ支援資金と挑戦支援資本強化特別貸付の融資限度額と返済期間の違いに加えて、要件や...,スタートアップ支援資金は融資限度額が20億円で直接貸付、返済期間は20年以内、要件はJVCA...,I don't know.,[page_content=' スタートアップの創業等を促進するため、日本政策金融公庫等に..., スタートアップの創業等を促進するため、日本政策金融公庫等において、創業等関連の融資・保証...,"{'reasoning': 'INCORRECT', 'value': 'INCORRECT...","{'reasoning': 'INCORRECT', 'value': 'INCORRECT...",0,0
6,宇宙戦略基金の設立と関連する技術開発テーマの具体的な支援分野について説明し、各分野間でどのよ...,宇宙戦略基金は、民間企業や大学、スタートアップ、国立研究機関に対して10年間にわたる研究開発...,The 宇宙戦略基金 (Space Strategy Fund) was establish...,[page_content='「宇宙戦略基金」\n■ 概要\n■ スキーム・要件\n■ 実績...,「宇宙戦略基金」\n■ 概要\n■ スキーム・要件\n■ 実績・アピールポイント\n予算額\...,"{'reasoning': 'CORRECT', 'value': 'CORRECT', '...","{'reasoning': 'CORRECT', 'value': 'CORRECT', '...",1,1
